In [1]:
import sqlite3
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import scale
import warnings
import pandas as pd 
import numpy as np
import sys
import matplotlib.pyplot as plt
import re
from sklearn.metrics import classification_report
import string
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

warnings.filterwarnings("ignore")
database = 'database.sqlite'

conn = sqlite3.connect(database)
tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)
print("Connection Successful",conn)

Connection Successful <sqlite3.Connection object at 0x000001EEA1A029D0>


In [2]:
country = pd.read_sql_query("SELECT * FROM Country", conn)
league = pd.read_sql_query("SELECT * FROM League", conn)
match = pd.read_sql_query("SELECT * FROM Match", conn)
player = pd.read_sql_query("SELECT * FROM Player", conn)
player_attributes = pd.read_sql_query("SELECT * FROM Player_Attributes", conn)
team = pd.read_sql_query("SELECT * FROM Team", conn)
team_attributes = pd.read_sql_query("SELECT * FROM Team_Attributes", conn)
sqlite_sequence = pd.read_sql_query("SELECT * FROM sqlite_sequence", conn)

country.set_index("id",inplace=True)
league.set_index("id",inplace=True)
match.set_index("id",inplace=True)
player.set_index("id",inplace=True)
player_attributes.set_index("id",inplace=True)
team.set_index("id",inplace=True)
team_attributes.set_index("id",inplace=True)
# sqlite_sequence.set_index("id",inplace=True)


In [3]:
match.drop(['goal', 'shoton',
       'shotoff', 'foulcommit', 'card', 'cross', 'corner', 'possession',
        'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA',
       'LBH', 'LBD', 'LBA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'SJH',
       'SJD', 'SJA', 'VCH', 'VCD', 'VCA', 'GBH', 'GBD', 'GBA', 'BSH', 'BSD',
       'BSA', 'home_player_X1', 'home_player_X2', 'home_player_X3',
       'home_player_X4', 'home_player_X5', 'home_player_X6', 'home_player_X7',
       'home_player_X8', 'home_player_X9', 'home_player_X10',
       'home_player_X11', 'away_player_X1', 'away_player_X2', 'away_player_X3',
       'away_player_X4', 'away_player_X5', 'away_player_X6', 'away_player_X7',
       'away_player_X8', 'away_player_X9', 'away_player_X10',
       'away_player_X11', 'home_player_Y1', 'home_player_Y2', 'home_player_Y3',
       'home_player_Y4', 'home_player_Y5', 'home_player_Y6', 'home_player_Y7',
       'home_player_Y8', 'home_player_Y9', 'home_player_Y10',
       'home_player_Y11', 'away_player_Y1', 'away_player_Y2', 'away_player_Y3',
       'away_player_Y4', 'away_player_Y5', 'away_player_Y6', 'away_player_Y7',
       'away_player_Y8', 'away_player_Y9', 'away_player_Y10',
       'away_player_Y11',],axis=1,inplace=True) 

In [4]:
match.sort_values(by="date",inplace=True)
match.head(1)

,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,...,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,B365H,B365D,B365A
id,,,,,,,,,,,,,,,,,,,,,
24559,24558,24558,2008/2009,1,2008-07-18 00:00:00,486263,10192,9931,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
def who_win(h_goal,a_goal):
    if h_goal>a_goal:
        return 1 # home team win

    elif h_goal < a_goal:
        return 2 # away team win
    
    else :
        return 0 #draw


In [6]:
match["label"] = match.apply(lambda x:who_win(x.home_team_goal,x.away_team_goal),axis=1)

In [7]:
all_matches=match[["home_team_api_id","away_team_api_id","date"]].values

In [ ]:
match["home_win_rate"] = 0
sayac = 0

dizi =np.array([])

for i in all_matches:
    x=match[(match["home_team_api_id"]==i[0]) & (match["away_team_api_id"]==i[1]) & (match["date"]<i[2])].label.value_counts()
    if (1 in x):
        dizi = np.append(dizi,x[1]/x.sum())
    elif x.empty: 
        dizi = np.append(dizi,"nan")
    elif (1 not in x) :
        dizi = np.append(dizi,0) 
    

In [ ]:
match["home_win_rate"] = dizi

In [ ]:
match.dropna(inplace=True)

In [ ]:
match["home_team_name"] = "nan"
for team_id in team.team_api_id.values:
    match["home_team_name"][match["home_team_api_id"]==team_id] = team[team["team_api_id"]==team_id]["team_long_name"].values[0]
match["away_team_name"] = "nan"
for team_id in team.team_api_id.values:
    match["away_team_name"][match["away_team_api_id"]==team_id] = team[team["team_api_id"]==team_id]["team_long_name"].values[0]

In [ ]:
player_overal = player_attributes[["player_fifa_api_id","player_api_id","date","overall_rating"]]
player_overal.dropna(inplace=True)

In [ ]:
player_overal[player_overal["date"]<"2009"].head()

In [ ]:
def date_to_year(date):
    return date[:4]

In [ ]:
player_overal["year"] = player_overal.date.apply(date_to_year)

In [ ]:
player_overal.drop_duplicates(subset = ["year","player_api_id"],keep="first",inplace = True)

In [ ]:
player_pos = ['home_player_1', 'home_player_2', 'home_player_3', 'home_player_4',
       'home_player_5', 'home_player_6', 'home_player_7', 'home_player_8',
       'home_player_9', 'home_player_10', 'home_player_11', 'away_player_1',
       'away_player_2', 'away_player_3', 'away_player_4', 'away_player_5',
       'away_player_6', 'away_player_7', 'away_player_8', 'away_player_9',
       'away_player_10', 'away_player_11']

In [ ]:
overals_2008 = dict()
overals_2009 = dict()
overals_2010 = dict()
overals_2011 = dict()
overals_2012 = dict()
overals_2013 = dict()
overals_2014 = dict()
overals_2015 = dict()
overals_2016 = dict()
overal_year_dict = dict()
for year in ["2008","2009","2010","2011","2012","2013","2014","2015","2016"]:
    print(year)
    for i in player_overal[player_overal["year"]==year]["player_api_id"].values:
        try :
            k=player_overal[(player_overal["player_api_id"]==i)&(player_overal["year"]==year)]["overall_rating"].values[0]
            if year=="2008":
                overals_2008.update({i:k})
            elif year=="2009":
                overals_2009.update({i:k})
            elif year=="2010":
                overals_2010.update({i:k})
            elif year=="2011":
                overals_2011.update({i:k})
            elif year=="2012":
                overals_2012.update({i:k})
            elif year=="2013":
                overals_2013.update({i:k})
            elif year=="2014":
                overals_2014.update({i:k})
            elif year=="2015":
                overals_2015.update({i:k})
            elif year=="2016":
                overals_2016.update({i:k})
        except Exception:
            pass
    
    
    
    

In [ ]:
import pickle

file = open("overals_2008.pickle","wb")
pickle.dump(overals_2008, file)
file.close()

file1 = open("overals_2009.pickle","wb")
pickle.dump(overals_2009, file1)
file1.close()

file2 = open("overals_2010.pickle","wb")
pickle.dump(overals_2010, file2)
file2.close()

file3 = open("overals_2011.pickle","wb")
pickle.dump(overals_2011, file3)
file3.close()

file4 = open("overals_2012.pickle","wb")
pickle.dump(overals_2012, file4)
file4.close()

file5 = open("overals_2013.pickle","wb")
pickle.dump(overals_2013, file5)
file5.close()

file6 = open("overals_2014.pickle","wb")
pickle.dump(overals_2014, file6)
file6.close()

file7 = open("overals_2015.pickle","wb")
pickle.dump(overals_2015, file7)
file7.close()


file8 = open("overals_2016.pickle","wb")
pickle.dump(overals_2016, file8)
file8.close()


In [ ]:
match["year"] = match.date.apply(date_to_year)
mean = int(np.floor(np.floor(player_overal["overall_rating"].mean())))

In [ ]:
for year in ["2008","2009","2010","2011","2012","2013","2014","2015","2016"]:
    i=0
    for pos in player_pos:
        try :
            if year == "2008":
                match.loc[match["year"]=="2008",pos] = match[pos][match["year"]=="2008"].apply(lambda x:overals_2008[x] 
                                                                                               if x in overals_2008.keys() else mean)
            
            elif year == "2009":
                match.loc[match["year"]=="2009",pos] = match[pos][match["year"]=="2009"].apply(lambda x:overals_2009[x] 
                                                                                               if x in overals_2009.keys() else mean)
            
            elif year == "2010":
                match.loc[match["year"]=="2010",pos] = match[pos][match["year"]=="2010"].apply(lambda x:overals_2010[x] 
                                                                                               if x in overals_2010.keys() else mean)
             
            elif year == "2011":
                match.loc[match["year"]=="2011",pos] = match[pos][match["year"]=="2011"].apply(lambda x:overals_2011[x] 
                                                                                               if x in overals_2011.keys() else mean)
             
            elif year == "2012":
                match.loc[match["year"]=="2012",pos] = match[pos][match["year"]=="2012"].apply(lambda x:overals_2012[x] 
                                                                                               if x in overals_2012.keys() else mean)
             
            elif year == "2013":
                match.loc[match["year"]=="2013",pos] = match[pos][match["year"]=="2013"].apply(lambda x:overals_2013[x] 
                                                                                               if x in overals_2013.keys() else mean)
             
            elif year == "2014":
                match.loc[match["year"]=="2014",pos] = match[pos][match["year"]=="2014"].apply(lambda x:overals_2014[x] 
                                                                                               if x in overals_2014.keys() else mean)
             
            elif year == "2015":
                match.loc[match["year"]=="2015",pos] = match[pos][match["year"]=="2015"].apply(lambda x:overals_2015[x] 
                                                                                               if x in overals_2015.keys() else mean)
            
            elif year == "2016":
                match.loc[match["year"]=="2016",pos] = match[pos][match["year"]=="2016"].apply(lambda x:overals_2016[x] 
                                                                                               if x in overals_2016.keys() else mean)
                
        except Exception:
            pass

In [ ]:
# match.to_csv("match4.csv",index=False)

In [ ]:
def bahis(h,d,a):
    if (h < d) & (h < a):
        return int(1)
    elif (a < h) &( a < d):
        return int(2)
    elif (d < h) & (d < a) :
        return int(0)

In [ ]:
match["bahis"] = match.apply(lambda x:bahis(x.B365H,x.B365D,x.B365A),axis=1)

In [ ]:
# match.to_csv("label.csv",index=False)


In [ ]:
# match["label"] = match.apply(lambda x:who_win(x.home_team_goal,x.away_team_goal),axis=1)
# match.to_csv("mat.csv",index=False)
match.tail()

In [ ]:
match.dropna(inplace=True)

In [ ]:
x = match.drop(["country_id","league_id","season","date","match_api_id","home_team_api_id","away_team_api_id",
              "home_team_goal","away_team_goal","stage","label","year","home_team_name","away_team_name"],axis=1).values

y = match["label"].values

In [ ]:
scaler = MinMaxScaler()
x_scaled=scaler.fit_transform(x)

x_train,x_test,y_train,y_test = train_test_split(x_scaled,y,test_size=.2,random_state= 41)
mnb =XGBClassifier()
mnb.fit(x_train,y_train)
mnb.score(x_test,y_test)

In [ ]:
mnb = MultinomialNB(alpha=0.01) # Çok sınıflı problemlerde kullanılır.
knn = KNeighborsClassifier(3)
svc = LinearSVC(random_state=44)
rfc = RandomForestClassifier(n_estimators=10,criterion='entropy',)#
tree = DecisionTreeClassifier()
sgd = SGDClassifier(alpha=0.01,random_state=44)
kmeans = KMeans(n_clusters=3, random_state=0)

In [ ]:
models=[mnb,svc,rfc,tree,sgd,knn]
best_acc=0
selected=models[0]
for count,model in enumerate(models):

    model=model.fit(x_train,y_train)
    accuracy=accuracy_score(y_test,model.predict(x_test))
    print(f"Model : {models[count]} \n Accuracy : {accuracy} \n")
    if accuracy>best_acc:
        best_acc=accuracy
        selected=model
#     except Exception:
#         continue
print("Best Accuracy :" + str(best_acc))
print("Selected Model : " + str(selected))

In [ ]:
# for i in player_pos[:-2]:
#     for player_id in player_unique:
#         df[i][df[i]==player_id]=player_attributes[player_attributes["player_api_id"]==player_id].iloc[0]["overall_rating"]

In [ ]:
# match.to_csv("denemeson.csv",index=False)